# Joining University Data and GeoID + County IDs using Location Crosswalk

### Progress: 

## Pulling University Data and Crosswalk Data From Box

In [38]:
from boxsdk import DevelopmentClient
import pandas as pd
import dotenv
import os
from io import BytesIO

In [39]:
env_location =  'C:\\Users\\trent\\Documents\\Capstone' ##change as necessaru
os.chdir(env_location)
dotenv.load_dotenv() # register the .env file where passwords are stored

True

In [112]:
CLIENT_ID = os.getenv("box_client_id")
CLIENT_SECRET = os.getenv("box_client_secret")
TOKEN = 'm9tDcDP8hOPvU92loJEXKRO8xm1VDE2q' ##copy and paste from dev

In [113]:
from boxsdk import OAuth2, Client

oauth = OAuth2(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    access_token=TOKEN,
)

box = Client(oauth)

me = box.user().get()

print('logged into Box as', me.login)

logged into Box as txc5ve@virginia.edu


In [42]:
MY_FOLDER_ID = 0
my_folder = box.folder(MY_FOLDER_ID).get()
my_folder
items = my_folder.get_items()
items
for i in items:
    print(i)

<Box Folder - 166970315293 (Patents22)>


In [44]:
universities_box_id = '988549931799'
universities_file = box.file(universities_box_id).get()

In [45]:
universities_content = box.file(universities_box_id).content()

In [46]:
universities = pd.read_csv(BytesIO(universities_content))

In [47]:
## crosswalk not used for join
crosswalk_box_id = '990881584176'
crosswalk_file = box.file(crosswalk_box_id).get()
crosswalk_file
crosswalk_content = box.file(crosswalk_box_id).content()
crosswalk = pd.read_csv(BytesIO(crosswalk_content))

C:\Users\trent\AppData\Local\Temp\ipykernel_34284\1472077329.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  crosswalk = pd.read_csv(BytesIO(crosswalk_content))


In [ ]:
# patents_box_id = '987961636758'
# patents_file = box.file(patents_box_id).get()
# patents_file
# patents_content = box.file(patents_box_id).content()
# patents = pd.read_csv(BytesIO(patents_content))

In [48]:
universities

,Unnamed: 0,Institution,Control,City,State,level
0,0,Arizona State University Campus Immersion,Public,Tempe,AZ,r1
1,1,Auburn University,Public,Auburn,AL,r1
2,2,Baylor University,Private (non-profit),Waco,TX,r1
3,3,Binghamton University,Public,Vestal,NY,r1
4,4,Boston College,Private (non-profit),Chestnut Hill,MA,r1
...,...,...,...,...,...,...
297,18,University of Texas Health Science Center at H...,Public,Houston,TX,r3
298,19,University of Texas Health Science Center at S...,Public,San Antonio,TX,r3
299,20,University of Texas Medical Branch,Public,Galveston,TX,r3
300,21,University of Texas Southwestern Medical Center,Public,Dallas,TX,r3


In [49]:
crosswalk.head(5)

,id,GEOID,pv_city,gl_city,pv_state,gl_state,country,pv_lat,gl_lat,pv_long,gl_long,pv_county,gl_county,pv_state_fips,pv_county_fips,gl_county_fips
0,f9139cb2-cb8f-11eb-9615-121df0c29c1e,6075,San Francisco,NaN,CA,NaN,US,37.7292,NaN,-123.0470,NaN,San Francisco,NaN,6.0,6075.0,NaN
1,ffc9f8bd-cb8e-11eb-9615-121df0c29c1e,32031,Reno,NaN,NV,NaN,US,39.5504,NaN,-119.8030,NaN,Washoe,NaN,32.0,32031.0,NaN
2,9596257a-cb90-11eb-9615-121df0c29c1e,50021,Rutland,NaN,VT,NaN,US,43.6106,NaN,-72.9726,NaN,Rutland,NaN,50.0,50021.0,NaN
3,499f8deb-cb8e-11eb-9615-121df0c29c1e,34003,Parkridge,Park Ridge,NJ,NJ,US,NaN,-74.03939,NaN,41.03379,NaN,Bergen,34.0,NaN,34003.0
4,ea59a211-09be-11ec-893a-12de62d610b1,6085,Sunnyvale,NaN,CA,NaN,US,37.3688,NaN,-122.0360,NaN,Santa Clara,NaN,6.0,6085.0,NaN


## Cleaning Data for Join on City/State

In [50]:
universities.City = universities.City.str.lower()
universities.State = universities.State.str.lower()
universities.dtypes

Unnamed: 0      int64
Institution    object
Control        object
City           object
State          object
level          object
dtype: object

In [51]:
##crosswalk ultimately not used due to the typos in city names
crosswalk.pv_city = crosswalk.pv_city.str.lower()
crosswalk.pv_state = crosswalk.pv_state.str.lower()
crosswalk.dtypes
crosswalk.query("pv_city == 'chestnut hill'")

,id,GEOID,pv_city,gl_city,pv_state,gl_state,country,pv_lat,gl_lat,pv_long,gl_long,pv_county,gl_county,pv_state_fips,pv_county_fips,gl_county_fips
2777,e297d56d-cb8e-11eb-9615-121df0c29c1e,25017,chestnut hill,Newton,ma,MA,US,42.3268,-71.17208,-71.1646,42.33295,NaN,Middlesex,25.0,NaN,25017.0


In [14]:
# patents.pv_city = patents.pv_city.str.lower()
# patents

## Joining on City and State

#### Fix Missing Cities first

In [52]:
universities['City']= universities['City'].replace('saint louis', 'st. louis')
universities['City']= universities['City'].replace('mount pleasant', 'mt. pleasant')

## this one listed wrong in wikipedia table, rio grande valley is not a city
universities['City']= universities['City'].replace('rio grande valley', 'edinburg')

### Join

In [53]:
university_geo_join = pd.merge(universities, crosswalk, left_on=['City', 'State'], right_on=['pv_city','pv_state'], how='left')

In [54]:
university_geo = university_geo_join[['Institution', 'Control', 'City', 'State', 'level', 'GEOID', 'pv_city', 'pv_state', 'pv_county']]

In [55]:
university_geo

,Institution,Control,City,State,level,GEOID,pv_city,pv_state,pv_county
0,Arizona State University Campus Immersion,Public,tempe,az,r1,4013,tempe,az,Maricopa
1,Auburn University,Public,auburn,al,r1,1081,auburn,al,Lee
2,Baylor University,Private (non-profit),waco,tx,r1,48309,waco,tx,McLennan
3,Binghamton University,Public,vestal,ny,r1,36007,vestal,ny,NaN
4,Boston College,Private (non-profit),chestnut hill,ma,r1,25017,chestnut hill,ma,NaN
...,...,...,...,...,...,...,...,...,...
308,University of Texas Health Science Center at H...,Public,houston,tx,r3,48201,houston,tx,Harris
309,University of Texas Health Science Center at S...,Public,san antonio,tx,r3,48029,san antonio,tx,Bexar
310,University of Texas Medical Branch,Public,galveston,tx,r3,48167,galveston,tx,Galveston
311,University of Texas Southwestern Medical Center,Public,dallas,tx,r3,48113,dallas,tx,Dallas


### Fix row that didn't join

In [56]:
university_geo[university_geo['GEOID'].isnull()]

,Institution,Control,City,State,level,GEOID,pv_city,pv_state,pv_county
222,Prairie View A&M University,Public,prairie view,tx,r2,NaN,NaN,NaN,NaN


In [57]:
###still an issue with prairie view, not in cross walk but county;'s geoid is below

crosswalk.loc[crosswalk.pv_city.str.contains('prairie')].query("pv_county == 'tx'")

,id,GEOID,pv_city,gl_city,pv_state,gl_state,country,pv_lat,gl_lat,pv_long,gl_long,pv_county,gl_county,pv_state_fips,pv_county_fips,gl_county_fips


In [58]:
crosswalk.query("pv_county == 'Waller'")

,id,GEOID,pv_city,gl_city,pv_state,gl_state,country,pv_lat,gl_lat,pv_long,gl_long,pv_county,gl_county,pv_state_fips,pv_county_fips,gl_county_fips
1565,3d3f18f9-cb8e-11eb-9615-121df0c29c1e,48473,waller,NaN,tx,NaN,US,30.0602,NaN,-95.9256,NaN,Waller,NaN,48.0,48473.0,NaN
2502,b41951e7-09bc-11ec-893a-12de62d610b1,48473,hempstead,NaN,tx,NaN,US,30.1053,NaN,-96.0805,NaN,Waller,NaN,48.0,48473.0,NaN
10167,449aac0c-cb8e-11eb-9615-121df0c29c1e,48473,brookshire,NaN,tx,NaN,US,29.7793,NaN,-95.9540,NaN,Waller,NaN,48.0,48473.0,NaN
18058,83e12478-cb8e-11eb-9615-121df0c29c1e,48473,pattison,NaN,tx,NaN,US,29.8213,NaN,-95.9859,NaN,Waller,NaN,48.0,48473.0,NaN


In [59]:
# university_geo = university_geo[
    
university_geo.GEOID.fillna("48473", inplace = True)
university_geo.query("City == 'prairie view'")

C:\Users\trent\AppData\Local\Temp\ipykernel_34284\2182127261.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  university_geo.GEOID.fillna("48473", inplace = True)


,Institution,Control,City,State,level,GEOID,pv_city,pv_state,pv_county
222,Prairie View A&M University,Public,prairie view,tx,r2,48473,NaN,NaN,NaN


In [60]:
university_geo.query("City == 'prairie view'")

,Institution,Control,City,State,level,GEOID,pv_city,pv_state,pv_county
222,Prairie View A&M University,Public,prairie view,tx,r2,48473,NaN,NaN,NaN


In [24]:
university_geo[university_geo['GEOID'].isnull()]

,Institution,Control,City,State,level,GEOID,pv_city,pv_state,pv_county


In [61]:
# university_geo.loc[university_geo.GEOID.str.contains('_')]

### Group GEOIDS and get university count
#### First need to one hot encode to get a numerical count for each level

In [62]:
def one_hot_encode(df, col):
    one_hot = pd.get_dummies(df[col])
    one_hot.columns = [col + '_' + str(x) for x in one_hot.columns]
    
#     df = df.drop(col, axis=1)
    df = df.join(one_hot)
    
    return df


In [63]:
university_geo_levels = one_hot_encode(university_geo, 'level')

In [195]:
university_geo_levels.Institution.value_counts()

##Some schools are repatings!! need to drop duplicates

Georgia State University                   2
University of Minnesota                    2
California State University, Long Beach    2
University of Delaware                     2
San Francisco State University             2
                                          ..
University of Miami                        1
University of Memphis                      1
University of Massachusetts Amherst        1
University of Maryland, College Park       1
Weill Cornell Medicine                     1
Name: Institution, Length: 302, dtype: int64

In [196]:
university_geo_levels.drop_duplicates(inplace = True)
university_geo_levels.value_counts()

Institution                          Control               City          State  level  GEOID  pv_city       pv_state  pv_county   level_r1  level_r2  level_r3
Albert Einstein College of Medicine  Private (non-profit)  bronx         ny     r3     36005  bronx         ny        Bronx       0         0         1           1
Tennessee State University           Public                nashville     tn     r2     47037  nashville     tn        Davidson    0         1         0           1
University of Illinois Chicago       Public                chicago       il     r1     17031  chicago       il        Cook        1         0         0           1
University of Idaho                  Public                moscow        id     r2     16057  moscow        id        Latah       0         1         0           1
University of Houston                Public                houston       tx     r1     48201  houston       tx        Harris      1         0         0           1
                     

In [197]:
geoids_universities = university_geo_levels.groupby(['GEOID']).agg({'level_r1':'sum','level_r2':'sum','level_r3':'sum'})

In [198]:
geoids_universities

,level_r1,level_r2,level_r3
GEOID,,,
1073,1,0,0
1081,1,0,0
1089,1,0,0
1097,0,1,0
1125,1,0,0
...,...,...,...
55079,1,1,1
56001,0,1,0
72127,0,1,0


In [200]:
geoids_universities.level_r1.value_counts()

1    112
0     92
2      7
3      5
5      1
Name: level_r1, dtype: int64

In [194]:
# university_geo.to_csv('test_uni.csv')

In [37]:
geoids_universities.to_csv('geoids_university_count.csv')

# The Final Merges! Universities, Ed Attainment, Patents

## Load in Ed Attainment and Patents

### Patents

In [146]:
patents_box_id = '992835605300'
patents_file = box.file(crosswalk_box_id).get()
patents_file
patents_content = box.file(patents_box_id).content()
patents = pd.read_csv(BytesIO(patents_content))

C:\Users\trent\AppData\Local\Temp\ipykernel_34284\292441299.py:5: DtypeWarning: Columns (0,6) have mixed types. Specify dtype option on import or set low_memory=False.
  patents = pd.read_csv(BytesIO(patents_content))


In [129]:
patents

,patent_number,assignee,grant_year,application_year,application_number,GEOID,ipc_section,team_size,inventors,men_inventors,...,ipc_P,ipc_Q,ipc_R,ipc_S,ipc_T,ipc_U,ipc_V,ipc_X,ipc_Y,ipc_Z
0,7472428,"NIKE, INC.",2009.0,2004,10814594,41067,A,2,2,2.0,...,0,0,0,0,0,0,0,0,0,0
1,7472496,"NIKE, INC.",2009.0,2005,11055158,41067,A,2,2,1.0,...,0,0,0,0,0,0,0,0,0,0
2,7472634,"SD3, LLC",2009.0,2004,10923290,41067,B,3,3,3.0,...,0,0,0,0,0,0,0,0,0,0
3,7474419,FEI COMPANY,2009.0,2006,11590583,41067,G,6,6,5.0,...,0,0,0,0,0,0,0,0,0,0
4,7474972,"TEKTRONIX, INC.",2009.0,2007,11690797,41067,G,3,3,3.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256090,NaN,King City Nursery LLC,NaN,2019,16424866,6053,NaN,1,1,1.0,...,0,0,0,0,0,0,0,0,0,0
2256091,NaN,JA SOLAR USA INC.,NaN,2019,16424879,6085,NaN,2,2,2.0,...,0,0,0,0,0,0,0,0,0,0
2256092,NaN,"SentriLock, LLC",NaN,2019,16424880,39061,NaN,1,1,1.0,...,0,0,0,0,0,0,0,0,0,0
2256093,NaN,"S.P.M. Flow Control, Inc.",NaN,2019,16424786,48439,NaN,1,1,1.0,...,0,0,0,0,0,0,0,0,0,0


### One Hot Encode ipc_section to aggregate

In [84]:
# patents.ipc_section.unique()

array(['A', 'B', 'G', 'H', 'G H', 'B H', 'F', 'A G', 'D', 'A B C', 'C H',
       'C', 'A B E', 'A B', nan, 'B F', 'B G H', 'F G', 'A E', 'B G',
       'A G H', 'A F', 'A D', 'F H', 'A H', 'E', 'B C', 'A C', 'A F H',
       'E F', 'A F G', 'A B D', 'B E', 'B D', 'B F G', 'B D G', 'F G H',
       'D H', 'C G', 'A B F', 'A B C H', 'B E F', 'B F G H', 'E G',
       'B C D', 'B C G', 'B C H', 'A B G', 'C F G', 'C G H', 'B F H',
       'C F', 'D G', 'B C F G', 'E H', 'A B H', 'A C H', 'B E F G H',
       'A D G', 'A B C D', 'B C G H', 'A C D', 'A F G H', 'A B G H',
       'A B D G H', 'A B C F G H', 'A C G', 'C Q', 'B C F G H', 'A B E H',
       'B E G', 'B C F', 'A E G', 'E F G', 'A C D F', 'A B C G',
       'A B D E', 'C D', 'B C E F', 'A B F G H', 'D G H', 'A E F H',
       'B D F H', 'C F G H', 'B C E', 'E G H', 'E F H', 'A B C G H',
       'A B E F', 'A C D G', 'C E', 'A C G H', 'A E H', 'B E F G',
       'A B C F', 'B E G H', 'C D F', 'C E G', 'B C F H', 'A E F',
       'B E F H', 'A B

In [ ]:
##way too many combos to one hot! will do it by letter

In [147]:
patents.ipc_section.fillna('None_Listed', inplace = True)
patents['no_section'] = [1 if a == 'None_Listed' else 0 for a in patents['ipc_section']]
patents

,patent_number,assignee,grant_year,application_year,application_number,GEOID,ipc_section,team_size,inventors,men_inventors,women_inventors,already_granted,no_section
0,7472428,"NIKE, INC.",2009.0,2004,10814594,41067,A,2,2,2.0,0.0,1,0
1,7472496,"NIKE, INC.",2009.0,2005,11055158,41067,A,2,2,1.0,1.0,1,0
2,7472634,"SD3, LLC",2009.0,2004,10923290,41067,B,3,3,3.0,0.0,1,0
3,7474419,FEI COMPANY,2009.0,2006,11590583,41067,G,6,6,5.0,1.0,1,0
4,7474972,"TEKTRONIX, INC.",2009.0,2007,11690797,41067,G,3,3,3.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256090,NaN,King City Nursery LLC,NaN,2019,16424866,6053,None_Listed,1,1,1.0,0.0,0,1
2256091,NaN,JA SOLAR USA INC.,NaN,2019,16424879,6085,None_Listed,2,2,2.0,0.0,0,1
2256092,NaN,"SentriLock, LLC",NaN,2019,16424880,39061,None_Listed,1,1,1.0,0.0,0,1
2256093,NaN,"S.P.M. Flow Control, Inc.",NaN,2019,16424786,48439,None_Listed,1,1,1.0,0.0,0,1


In [148]:
list = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
for x in list:
    ipc = 'ipc_' + x
    patents[ipc] = [1 if x in str(a) else 0 for a in patents['ipc_section']]
patents

,patent_number,assignee,grant_year,application_year,application_number,GEOID,ipc_section,team_size,inventors,men_inventors,...,ipc_Q,ipc_R,ipc_S,ipc_T,ipc_U,ipc_V,ipc_W,ipc_X,ipc_Y,ipc_Z
0,7472428,"NIKE, INC.",2009.0,2004,10814594,41067,A,2,2,2.0,...,0,0,0,0,0,0,0,0,0,0
1,7472496,"NIKE, INC.",2009.0,2005,11055158,41067,A,2,2,1.0,...,0,0,0,0,0,0,0,0,0,0
2,7472634,"SD3, LLC",2009.0,2004,10923290,41067,B,3,3,3.0,...,0,0,0,0,0,0,0,0,0,0
3,7474419,FEI COMPANY,2009.0,2006,11590583,41067,G,6,6,5.0,...,0,0,0,0,0,0,0,0,0,0
4,7474972,"TEKTRONIX, INC.",2009.0,2007,11690797,41067,G,3,3,3.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256090,NaN,King City Nursery LLC,NaN,2019,16424866,6053,None_Listed,1,1,1.0,...,0,0,0,0,0,0,0,0,0,0
2256091,NaN,JA SOLAR USA INC.,NaN,2019,16424879,6085,None_Listed,2,2,2.0,...,0,0,0,0,0,0,0,0,0,0
2256092,NaN,"SentriLock, LLC",NaN,2019,16424880,39061,None_Listed,1,1,1.0,...,0,0,0,0,0,0,0,0,0,0
2256093,NaN,"S.P.M. Flow Control, Inc.",NaN,2019,16424786,48439,None_Listed,1,1,1.0,...,0,0,0,0,0,0,0,0,0,0


In [159]:
patents['ipc_Z'].value_counts()
patents['ipc_Y'].value_counts()
patents['ipc_X'].value_counts()
patents['ipc_W'].value_counts()
patents['ipc_V'].value_counts()
patents['ipc_U'].value_counts()
patents['ipc_T'].value_counts()
patents['ipc_S'].value_counts()
patents['ipc_R'].value_counts()
patents['ipc_Q'].value_counts()
patents['ipc_P'].value_counts()
patents = patents.drop(columns = ['ipc_Z','ipc_X','ipc_W','ipc_U','ipc_T'])

In [160]:
patents

,patent_number,assignee,grant_year,application_year,application_number,GEOID,ipc_section,team_size,inventors,men_inventors,...,ipc_L,ipc_M,ipc_N,ipc_O,ipc_P,ipc_Q,ipc_R,ipc_S,ipc_V,ipc_Y
0,7472428,"NIKE, INC.",2009.0,2004,10814594,41067,A,2,2,2.0,...,0,0,0,0,0,0,0,0,0,0
1,7472496,"NIKE, INC.",2009.0,2005,11055158,41067,A,2,2,1.0,...,0,0,0,0,0,0,0,0,0,0
2,7472634,"SD3, LLC",2009.0,2004,10923290,41067,B,3,3,3.0,...,0,0,0,0,0,0,0,0,0,0
3,7474419,FEI COMPANY,2009.0,2006,11590583,41067,G,6,6,5.0,...,0,0,0,0,0,0,0,0,0,0
4,7474972,"TEKTRONIX, INC.",2009.0,2007,11690797,41067,G,3,3,3.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256090,NaN,King City Nursery LLC,NaN,2019,16424866,6053,None_Listed,1,1,1.0,...,1,0,1,0,0,0,0,0,0,0
2256091,NaN,JA SOLAR USA INC.,NaN,2019,16424879,6085,None_Listed,2,2,2.0,...,1,0,1,0,0,0,0,0,0,0
2256092,NaN,"SentriLock, LLC",NaN,2019,16424880,39061,None_Listed,1,1,1.0,...,1,0,1,0,0,0,0,0,0,0
2256093,NaN,"S.P.M. Flow Control, Inc.",NaN,2019,16424786,48439,None_Listed,1,1,1.0,...,1,0,1,0,0,0,0,0,0,0


### Aggregate Patents by GEOID and App Year

In [168]:
patents_geoid_year = patents.groupby(['GEOID','application_year']).agg({'team_size':'mean',
                                                   'inventors':'mean',
                                                   'men_inventors':'mean',
                                                   'women_inventors':'mean',
                                                   'already_granted':'mean',
                                                  'ipc_A':'mean',
                                                  'ipc_B':'mean',
                                                  'ipc_C':'mean',
                                                  'ipc_D':'mean',
                                                  'ipc_E':'mean',
                                                  'ipc_F':'mean',
                                                  'ipc_G':'mean',
                                                  'ipc_H':'mean',
                                                  'ipc_I':'mean',
                                                  'ipc_J':'mean',
                                                  'ipc_K':'mean',
                                                  'ipc_L':'mean',
                                                  'ipc_M':'mean',
                                                  'ipc_N':'mean',
                                                  'ipc_O':'mean',
                                                  'ipc_P':'mean',
                                                  'ipc_Q':'mean',
                                                  'ipc_R':'mean',
                                                  'ipc_S':'mean',
                                                  'ipc_V':'mean',
                                                  'ipc_Y':'mean',}).reset_index()

In [169]:
patents_geoid_year

,GEOID,application_year,team_size,inventors,men_inventors,women_inventors,already_granted,ipc_A,ipc_B,ipc_C,...,ipc_L,ipc_M,ipc_N,ipc_O,ipc_P,ipc_Q,ipc_R,ipc_S,ipc_V,ipc_Y
0,1003,2005,1.000000,1.000000,1.000000,0.000000,1.000000,0.500000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1003,2006,1.000000,1.000000,0.500000,0.500000,1.000000,0.500000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1003,2008,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1003,2009,1.000000,1.000000,1.000000,0.000000,0.500000,0.500000,0.0,0.0,...,0.500000,0.0,0.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1003,2010,1.636364,1.636364,1.636364,0.000000,0.545455,0.454545,0.0,0.0,...,0.454545,0.0,0.454545,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22105,78030,2014,1.800000,1.800000,1.600000,0.200000,0.600000,0.200000,0.0,0.0,...,0.600000,0.0,0.600000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22106,78030,2015,1.666667,1.666667,1.333333,0.333333,0.666667,0.000000,0.0,0.0,...,0.666667,0.0,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22107,78030,2016,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.0,0.0,...,1.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22108,78030,2017,1.333333,1.333333,1.333333,0.000000,1.000000,0.333333,0.0,0.0,...,0.333333,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Ed Attainment

In [165]:
ed_box_id = '992854565190'
ed_file = box.file(ed_box_id).get()
ed_file
ed_content = box.file(ed_box_id).content()
ed_attainment = pd.read_csv(BytesIO(ed_content))

In [166]:
ed_attainment

,Unnamed: 0,GEO_ID_CLEAN,GEO_ID,NAME,year,Perc_Over25_LessNinthGrade,Perc_Over25_SomeHighSchool,Perc_Over25_HighSchoolGrad,Perc_Over25_SomeCollege,Perc_Over25_Assosciates,Perc_Over25_Bachelors,Perc_Over25_Graduate
0,1,1001,0500000US01001,"Autauga County, Alabama",2010,5.1,9.4,34.4,22.6,5.0,15.3,8.2
1,2,1003,0500000US01003,"Baldwin County, Alabama",2010,4.3,9.5,29.4,22.3,5.9,19.8,8.9
2,3,1005,0500000US01005,"Barbour County, Alabama",2010,8.5,21.0,36.1,16.6,5.2,6.9,5.7
3,4,1007,0500000US01007,"Bibb County, Alabama",2010,10.4,16.6,43.7,13.4,5.6,8.4,1.8
4,5,1009,0500000US01009,"Blount County, Alabama",2010,9.9,16.0,37.1,19.5,4.6,9.0,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...
106366,106399,72145,0500000US72145,"Vega Baja Municipio, Puerto Rico",2020,16.2,10.3,28.5,12.0,11.4,15.6,6.1
106367,106400,72147,0500000US72147,"Vieques Municipio, Puerto Rico",2020,19.8,9.5,45.4,2.8,9.4,7.2,5.8
106368,106401,72149,0500000US72149,"Villalba Municipio, Puerto Rico",2020,16.0,5.5,37.4,11.2,10.1,16.0,3.7
106369,106402,72151,0500000US72151,"Yabucoa Municipio, Puerto Rico",2020,17.1,11.0,25.9,16.4,12.0,15.0,2.6


In [212]:
ed_attainment[ed_attainment.GEO_ID_CLEAN.isnull()]

,Unnamed: 0,GEO_ID_CLEAN,GEO_ID,NAME,year,Perc_Over25_LessNinthGrade,Perc_Over25_SomeHighSchool,Perc_Over25_HighSchoolGrad,Perc_Over25_SomeCollege,Perc_Over25_Assosciates,Perc_Over25_Bachelors,Perc_Over25_Graduate


#### Need to group GEO_IDs

In [217]:
ed_attainment_agg = ed_attainment.groupby(['GEO_ID_CLEAN', 'year']).mean().reset_index().drop(columns = 'Unnamed: 0')

In [218]:
ed_attainment_agg

,GEO_ID_CLEAN,year,Perc_Over25_LessNinthGrade,Perc_Over25_SomeHighSchool,Perc_Over25_HighSchoolGrad,Perc_Over25_SomeCollege,Perc_Over25_Assosciates,Perc_Over25_Bachelors,Perc_Over25_Graduate
0,1001,2010,5.1,9.4,34.4,22.6,5.0,15.3,8.2
1,1001,2011,4.9,9.0,33.6,22.9,6.0,15.9,7.7
2,1001,2012,5.8,9.2,32.0,23.1,5.4,16.3,8.2
3,1001,2013,5.7,9.5,32.4,24.1,5.7,14.8,7.8
4,1001,2014,4.8,9.6,32.0,22.8,6.9,14.6,9.4
...,...,...,...,...,...,...,...,...,...
35452,72153,2019,19.0,7.6,33.3,8.1,7.4,19.0,5.7
35453,72153,2020,16.6,7.6,34.7,8.3,8.4,18.9,5.4
35454,78010,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35455,78020,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Merge University GeoID info to Patents

In [202]:
geoids_universities.loc[geoids_universities.level_r1 > 4]

,level_r1,level_r2,level_r3
GEOID,,,
25017,5,1,0


In [203]:
merged_patents_uni = pd.merge(patents_geoid_year, geoids_universities, on = 'GEOID', how='left', 
                       indicator='matched', validate='many_to_one')
merged_patents_uni

,GEOID,application_year,team_size,inventors,men_inventors,women_inventors,already_granted,ipc_A,ipc_B,ipc_C,...,ipc_P,ipc_Q,ipc_R,ipc_S,ipc_V,ipc_Y,level_r1,level_r2,level_r3,matched
0,1003,2005,1.000000,1.000000,1.000000,0.000000,1.000000,0.500000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,left_only
1,1003,2006,1.000000,1.000000,0.500000,0.500000,1.000000,0.500000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,left_only
2,1003,2008,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,left_only
3,1003,2009,1.000000,1.000000,1.000000,0.000000,0.500000,0.500000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,left_only
4,1003,2010,1.636364,1.636364,1.636364,0.000000,0.545455,0.454545,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22105,78030,2014,1.800000,1.800000,1.600000,0.200000,0.600000,0.200000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,left_only
22106,78030,2015,1.666667,1.666667,1.333333,0.333333,0.666667,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,left_only
22107,78030,2016,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,left_only
22108,78030,2017,1.333333,1.333333,1.333333,0.000000,1.000000,0.333333,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,left_only


In [204]:
merged_patents_uni.matched.value_counts()

left_only     18014
both           4096
right_only        0
Name: matched, dtype: int64

In [205]:
merged_patents_uni.level_r1.value_counts()

1.0    2221
0.0    1547
2.0     168
3.0     126
5.0      34
Name: level_r1, dtype: int64

### Merge Ed Attainment info to Patents_Univ

In [220]:
merged_patents_uni_ed = pd.merge(merged_patents_uni.drop(columns ='matched'), ed_attainment_agg, left_on = ['GEOID','application_year'], right_on = ['GEO_ID_CLEAN','year'], how='left', 
                       indicator='matched', validate='one_to_one')
merged_patents_uni_ed

,GEOID,application_year,team_size,inventors,men_inventors,women_inventors,already_granted,ipc_A,ipc_B,ipc_C,...,GEO_ID_CLEAN,year,Perc_Over25_LessNinthGrade,Perc_Over25_SomeHighSchool,Perc_Over25_HighSchoolGrad,Perc_Over25_SomeCollege,Perc_Over25_Assosciates,Perc_Over25_Bachelors,Perc_Over25_Graduate,matched
0,1003,2005,1.000000,1.000000,1.000000,0.000000,1.000000,0.500000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,1003,2006,1.000000,1.000000,0.500000,0.500000,1.000000,0.500000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,1003,2008,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,1003,2009,1.000000,1.000000,1.000000,0.000000,0.500000,0.500000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,1003,2010,1.636364,1.636364,1.636364,0.000000,0.545455,0.454545,0.0,0.0,...,1003.0,2010.0,4.3,9.5,29.4,22.3,5.9,19.8,8.9,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22105,78030,2014,1.800000,1.800000,1.600000,0.200000,0.600000,0.200000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
22106,78030,2015,1.666667,1.666667,1.333333,0.333333,0.666667,0.000000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
22107,78030,2016,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
22108,78030,2017,1.333333,1.333333,1.333333,0.000000,1.000000,0.333333,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [221]:
merged_patents_uni_ed.matched.value_counts()

both          13819
left_only      8291
right_only        0
Name: matched, dtype: int64